In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.demand_forecast_by_week 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH
max_proc_date AS (
    SELECT week, year, MAX(proc_date) max_proc_date FROM {catalog_name}.udp_wcm_silver_dataportal.demand_forecast_by_week GROUP BY week, year
)
SELECT a.* FROM {catalog_name}.udp_wcm_silver_dataportal.demand_forecast_by_week a
INNER JOIN max_proc_date b ON a.week = b.week AND a.year = b.year AND a.proc_date = b.max_proc_date;
""")